In [2]:
class Graph:
    def __init__(self):
        self.adj_dict = {}
        self.node_dict = {}
        self.edge_list = []
        
    def add_edge(self, v, u, pattern):
        if v not in self.adj_dict:
            self.adj_dict[v] = {u:pattern}
        else:
            self.adj_dict[v][u] = pattern
            
        if u not in self.adj_dict:
            self.adj_dict[u] = {v:pattern}
        else:
            self.adj_dict[u][v] = pattern
            
        if v not in self.node_dict:
            self.node_dict[v] = v
            
        if u not in self.node_dict:
            self.node_dict[u] = u
        
        self.edge_list.append([v,u])
        
    def del_edge(self, v, u):
        if u in self.adj_dict[v]:
            del(self.adj_dict[v][u])
            
        if v in self.adj_dict[u]:
            del(self.adj_dict[u][v])
            
    def del_uedge(self, v, u):
        if v in self.adj_dict[u]:
            del(self.adj_dict[u][v])
            
            
    def print_edge(self):
        return self.edge_list
        
    def print_adj(self, v):
        return self.adj_dict[v]
    
    def vertex(self, v):
        self.adj_list = self.adj_dict[v]
        self.deg = len(self.adj_dict[v])
        self.vertex_num = 1
        
        return self.vertex_num, self.deg, self.adj_list

In [3]:
import sys
import time

# edges_path = './dataset/soc-YouTube-ASU/soc-YouTube-ASU.edges' #sys.argv[1]
# label_path = './dataset/soc-YouTube-ASU/soc-YouTube-ASU.node_labels' #sys.argv[2]
# edges_path = './dataset/ENZYMES118/ENZYMES118.edges' #sys.argv[1]
# label_path = './dataset/ENZYMES118/ENZYMES118.node_labels' #sys.argv[2]
edges_path = './dataset/dblp/dblp.txt' #sys.argv[1]
label_path = './dataset/dblp/node2label.txt' #sys.argv[2]
label_dict = {}
graph_sort = []
label_kind = []
node_dict = {}
pt_fq = {} # 빈발패턴
label_f = open(label_path, 'r')
while True:
    line = label_f.readline().replace('\n', '').strip()
    if not line:
        label_kind.append('0')
        break
    
    node = line.split(' ')[0]
    label = line.split(' ')[1]
    label_dict[node] = label
    label_kind.append(label)
label_f.close

label_kind = list(set(label_kind))

edge_f = open(edges_path, 'r')
gs = Graph()
while True:
    line = edge_f.readline().replace('\n', '').strip()
    if not line:
        break
    
    v = line.split(' ')[0]
    u = line.split(' ')[1]
    
    if v == u:
        continue
    
    if v not in label_dict:
        label_dict[v] = '0'
        
    if u not in label_dict:
        label_dict[u] = '0'
    graph_sort.append([v,u])
    
    if v not in node_dict:
        node_dict[v] = v
    if u not in node_dict:
        node_dict[u] = u
        
    pt = label_dict[v] + '/' + label_dict[u]
    re_pt = ''.join(reversed(pt))
    
    if pt not in pt_fq and re_pt not in pt_fq:
        pt_fq[pt] = 1
        gs.add_edge(v,u,pt)
        
    elif re_pt in pt_fq:
        pt_fq[re_pt] = pt_fq[re_pt] + 1
        gs.add_edge(v,u,re_pt)
        
    elif pt in pt_fq:
        pt_fq[pt] = pt_fq[pt] + 1
        gs.add_edge(v,u,pt)

edge_f.close

new_pt_fq = {}
new_pt_fq.update(pt_fq)

In [4]:
p_id = 0
Freq = 0
TF = 0
FR = 0

def Top_nPercent(dict, n):
    if len(dict) == 0:
        return 0
    part_v = len(dict)*n//100 - 1
    value = list(sorted(dict.items(), key = lambda item: item[1], reverse = True))[part_v][1]
    return value

def pattern_gen(ptternFrequency, pttern_th):
    add_pt = {}
    
    for pt1 in ptternFrequency:
        if ptternFrequency[pt1] >= pttern_th:
            pt_tgt = pt1.split('/')
            for pt2 in ptternFrequency:
                if pt1 != pt2 and ptternFrequency[pt2] >= pttern_th:
                    pt_sce = pt2.split('/')
                    between_len = len(pt_tgt) - 2
                    if between_len == 0:
                        if pt_tgt[0] == pt_sce[0]:
                            pt = pt_sce[1] + '/' + pt1
                        elif pt_tgt[0] == pt_sce[1]:
                            pt = pt_sce[0] + '/' + pt1
                        elif pt_tgt[-1] == pt_sce[0]:
                            pt = pt1 + '/' + pt_sce[1]

                        elif pt_tgt[-1] == pt_sce[-1]:
                            pt = pt1 + '/' + pt_sce[0]
                        else:
                            continue
                    else:
                        if pt_tgt[:between_len] == pt_sce[between_len:-1]: #앞, 뒤
                            pt = pt_sce[0] + '/' + pt1
                        elif pt_tgt[between_len:-1] == pt_sce[:between_len]: #뒤, 앞
                            pt = pt1 + '/' + pt_sce[-1]
                        else:
                            continue
                        
                    re_pt = ''.join(reversed(pt))
        
                    if pt not in add_pt and re_pt not in add_pt:
                        add_pt[pt] = min(ptternFrequency[pt1], ptternFrequency[pt2])
    
    return add_pt

pers = 100
def updatePattern(updateDict, pers):
    update_dict = (pattern_gen(updateDict, Top_nPercent(updateDict, pers)))
    if len(update_dict) != 0:
        new_pt_fq.update(update_dict)
        updatePattern(update_dict, pers)
    
updatePattern(pt_fq, pers)
print(len(pt_fq))
print(len(new_pt_fq))
    
# print('a')
# a = pattern_gen(pt_fq, Top_nPercent(pt_fq, pers))
# print(len(a), Top_nPercent(pt_fq, pers))
# print('b')
# b = pattern_gen(a, Top_nPercent(a, pers))
# print(len(b), Top_nPercent(a, pers))
# print('c')
# c = pattern_gen(b, Top_nPercent(b, pers))
# print(len(c), Top_nPercent(b, pers))

# print(sorted(b.items(), key = lambda item: item[1], reverse = True))
            

55
4956


In [100]:
# print(list(sorted(node_dict.items(), key = lambda item: item[1], reverse = True))[:10])
# print(list(sorted(node_dict.items(), key = lambda item: item[1]))[:10])
# print(list(sorted(new_pt_fq.items(), key = lambda item: item[1], reverse = True))[:10])
# print(list(sorted(new_pt_fq.items(), key = lambda item: item[1]))[:10])

In [5]:
def pattern_Marge(v, pt):
    score = {}
    
    for u in pt:
        re_pt = ''.join(reversed(pt[u]))
        if pt[u] in new_pt_fq.keys() or re_pt in new_pt_fq.keys():
            deg_max = gs.vertex(v)[1]
            same_deg = len((pt.keys() & gs.vertex(u)[2].keys()))
            deg_ratio = gs.vertex(u)[1]/gs.vertex(v)[1]
            
            score[u] = [same_deg/deg_max + deg_ratio, pt[u]]
    
        gs.del_uedge(v, u)
    if score:
        su = list(sorted(score.items(), key = lambda item: item[1], reverse = True))[0]
        gs.del_edge(v, su[0])
        MargeNode = v + '/' + su[0]
        return MargeNode, su[1][1]
    else:
        return 0, 0
    
def pattern_Marge2(MargeNode, pt):
    frontV = MargeNode.split('/')[0]
    backV = MargeNode.split('/')[-1]
    union_list1 = {}
    union_list2 = {}
    frontScore = {}
    backScore = {}

    for front in gs.vertex(frontV)[2].keys():
        if backV ==  front:
            continue
        union_list1[front] = label_dict[front] + '/' + pt
        
    for back in gs.vertex(backV)[2].keys():
        if frontV == back:
            continue
        union_list2[back] = pt + '/' + label_dict[back]
            
    for union in union_list1.keys():
        re_pt = ''.join(reversed(union_list1[union]))
        
        if union_list1[union] in new_pt_fq.keys() or re_pt in new_pt_fq.keys():
            deg_max = gs.vertex(frontV)[1]
            same_deg = len((gs.vertex(frontV)[2].keys() & gs.vertex(union)[2].keys()))
            deg_ratio = gs.vertex(union)[1]/gs.vertex(frontV)[1]
            
            frontScore[union] = same_deg/deg_max + deg_ratio
        
        gs.del_uedge(frontV, union)
            
    for union in union_list2.keys():
        re_pt = ''.join(reversed(union_list2[union]))
        if union_list2[union] in new_pt_fq.keys() or re_pt in new_pt_fq.keys():
            deg_max = gs.vertex(backV)[1]
            same_deg = len((gs.vertex(backV)[2].keys() & gs.vertex(union)[2].keys()))
            deg_ratio = gs.vertex(union)[1]/gs.vertex(backV)[1]
            
            backScore[union] = same_deg/deg_max + deg_ratio
            
        gs.del_uedge(backV, union)

    if frontScore or backScore:
        if frontScore:
            frontMax = list(sorted(frontScore.items(), key = lambda item: item[1], reverse = True))[0]
        else:
            frontMax = [0,0]
            
        if backScore:
            backMax = list(sorted(backScore.items(), key = lambda item: item[1], reverse = True))[0]
        else:
            backMax = [0,0]
            
        if frontMax[1] >= backMax[1] and frontMax[1] != 0:
            gs.del_edge(frontV, frontMax[0])
            MargeNode = frontMax[0] + '/' + MargeNode
            new_pt = union_list1[frontMax[0]]
            
        elif frontMax[1] < backMax[1] and backMax[1] != 0:
            gs.del_edge(backV, backMax[0])
            MargeNode = MargeNode + '/' + backMax[0]
            new_pt = union_list2[backMax[0]]
            
        else:
            return MargeNode, pt, 0
            
        return MargeNode, new_pt, 1
    
    else:
        return MargeNode, pt, 0


In [6]:
for n in node_dict:
    node_dict[n] = gs.vertex(n)[1]
    
nodeList = []
node_list = sorted(node_dict.items(), key = lambda item: item[1], reverse = True)
for n in node_list:
    nodeList.append(n[0])


In [7]:
pattern_dict = {}
ZERONode = []
allEdgeDict = {}
while(True):
    edgeList = []
    node_size = len(nodeList)
    
    if node_size <= 0: #48
        break
    
    node = nodeList[0]
    print(node)
    s1 = time.time()
    newV, mP = pattern_Marge(node, gs.vertex(node)[2])
    
    if newV == 0:
        nodeList.remove(node)
        ZERONode.append(node)
        print(f'ZERONode: {node}')
        continue
    
    boolz = 1
    while (boolz > 0):
        newV, mP, boolz = pattern_Marge2(newV, mP)
    
    del_list = newV.split('/')
    for del_one in del_list:
        nodeList.remove(del_one)
        edge_list = list(gs.vertex(del_one)[2].keys())                
        edgeList.extend(edge_list)
        allEdgeDict[del_one] = newV
    
    if mP not in pattern_dict:
        pattern_dict[mP] = {newV:edgeList}
        
    else:
        pattern_dict[mP][newV] = edgeList

    
    e1 = time.time()
    d1 = e1-s1
    print(node_size, d1)
    
    

992
28085 0.008939981460571289
5388
28081 0.005957365036010742
20855
28077 0.0044708251953125
3345
28073 0.0034761428833007812
2356
28069 0.003476858139038086
5062
28065 0.003476381301879883
5134
28061 0.002483367919921875
159
28057 0.0029840469360351562
2691
28053 0.0029764175415039062
806
28049 0.0019865036010742188
1491
28045 0.003169536590576172
4878
28041 0.001993417739868164
20796
28037 0.001993417739868164
3742
28033 0.001992940902709961
1069
28029 0.001993417739868164
2680
28025 0.0019931793212890625
41
28021 0.0019919872283935547
17479
28017 0.0019958019256591797
6956
28013 0.002989053726196289
465
28009 0.0009965896606445312
3653
28005 0.0019931793212890625
17493
28001 0.0009965896606445312
1531
27997 0.0019936561584472656
485
27993 0.0019915103912353516
6311
27989 0.0020589828491210938
3595
27985 0.0021209716796875
5115
27981 0.0019931793212890625
1267
27977 0.0009968280792236328
2797
27973 0.0019931793212890625
21311
27969 0.0009963512420654297
6787
27965 0.0019934177398681

In [ ]:
print(len(pattern_dict))
print(pattern_dict)
print()

In [ ]:
print(new_pt_fq)

In [106]:
for pd in pattern_dict:
    for pd2 in pattern_dict[pd]:
        for pd3 in pattern_dict[pd][pd2]:
            if pd3 in allEdgeDict :
                
                pattern_dict[pd][pd2].remove(pd3)
                pattern_dict[pd][pd2].append(allEdgeDict[pd3])
print(pattern_dict)


{'1/1': {'54/36': ['27', '53', '35', '17/18', '45/44/88', '17/18'], '26/9': ['27', '8/60'], '52/51': ['50'], '17/18': []}, '2/2/1/1': {'62/39/55/3': ['63', '83', '21', '56', '20', '12/11/64/65', '85/38/84', '40/87', '2/57', '59/4'], '96/61/7/24': ['68', '95', '6', '23', '43', '52/51', '74/75/25/42', '8/60', '74/75/25/42'], '74/75/25/42': ['6', '41', '26/9', '43/89']}, '1/1/2/2': {'20/21/69/70': ['71', '20/21/69/70', '5/22/73'], '15/16/68/67': ['14', '52', '33', '17', '34/82', '52/51'], '12/11/64/65': ['63', '10'], '30/29/77/76': ['80', '30/29/77/76', '79/78']}, '1/1/2': {'5/22/73': ['6', '23', '59/4', '40/87'], '32/13/66': ['33', '14', '80/31', '49/94/48', '12/11/64/65'], '45/44/88': []}, '2/1': {'2/57': ['56', '58/19'], '40/87': ['41'], '34/82': ['33', '35/81'], '43/89': ['45/44/88'], '59/4': [], '8/60': [], '58/19': ['72/71'], '80/31': [], '35/81': [], '86/37': [], '93/46': [], '53/95': []}, '2/1/1': {'49/94/48': ['50', '90', '92/47/91']}, '2/2/1': {'85/38/84': ['83', '86/37'], '92/4

In [116]:
pEdgeList = []
pNodeDict = {}

for pd in pattern_dict:
    for pd2 in pattern_dict[pd]:
        if pd2 not in node_dict:
            node_dict[pd2] = pd2
        for pd3 in pattern_dict[pd][pd2]:
            pEdgeList.append([pd2, pd3])
            if pd3 not in node_dict:
                node_dict[pd3] = pd3

print(pEdgeList)

[['54/36', '27'], ['54/36', '53'], ['54/36', '35'], ['54/36', '17/18'], ['54/36', '45/44/88'], ['54/36', '17/18'], ['26/9', '27'], ['26/9', '8/60'], ['52/51', '50'], ['62/39/55/3', '63'], ['62/39/55/3', '83'], ['62/39/55/3', '21'], ['62/39/55/3', '56'], ['62/39/55/3', '20'], ['62/39/55/3', '12/11/64/65'], ['62/39/55/3', '85/38/84'], ['62/39/55/3', '40/87'], ['62/39/55/3', '2/57'], ['62/39/55/3', '59/4'], ['96/61/7/24', '68'], ['96/61/7/24', '95'], ['96/61/7/24', '6'], ['96/61/7/24', '23'], ['96/61/7/24', '43'], ['96/61/7/24', '52/51'], ['96/61/7/24', '74/75/25/42'], ['96/61/7/24', '8/60'], ['96/61/7/24', '74/75/25/42'], ['74/75/25/42', '6'], ['74/75/25/42', '41'], ['74/75/25/42', '26/9'], ['74/75/25/42', '43/89'], ['20/21/69/70', '71'], ['20/21/69/70', '20/21/69/70'], ['20/21/69/70', '5/22/73'], ['15/16/68/67', '14'], ['15/16/68/67', '52'], ['15/16/68/67', '33'], ['15/16/68/67', '17'], ['15/16/68/67', '34/82'], ['15/16/68/67', '52/51'], ['12/11/64/65', '63'], ['12/11/64/65', '10'], ['3

In [117]:
import networkx as nx
from pyvis.network import Network
G = nx.Graph()

for v in pNodeDict:
    G.add_node(v, title = v)
    
for v, u in pEdgeList:
    G.add_edge(v,u)
    
graph_name = 'ENZYMES118_pattern' + '.html'
net = Network(bgcolor="#222222", font_color="white", filter_menu=True, select_menu=True)
net.from_nx(G)

net.show_buttons(filter_=['physics'])
net.show(graph_name)